In [2]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as mlp
import matplotlib.pyplot as plt
import geopandas as gp
import os

%matplotlib inline
plt.style.use('ggplot')
mlp.rcParams['figure.figsize'] = [8.0, 8.0]

In [3]:
base_dir = "../../../../Outcome Measurement Data/MALARIA/Distribucion de MTILD/"

In [4]:
entregas = None
entregasDetalles = None
for year in [2015, 2016, 2017, 2018]:
    for (dirpath, dirnames, filenames) in os.walk(base_dir + "BD " + str(year) + "/"):
        for filename in filenames:
            if "EntregaPIIs." in filename:
                if (filename.endswith(".csv")):
                    temp = pd.read_csv(dirpath + "/" + filename)
                elif (filename.endswith(".xlsx")):
                    temp = pd.read_excel(dirpath + "/" + filename, 0)
                temp["bd_year"] = year
                if entregas is None:
                    entregas = temp
                else:
                    entregas = pd.concat([entregas, temp]) 
            elif "EntregaPIIsDetalle." in filename:
                if (filename.endswith(".csv")):
                    temp = pd.read_csv(dirpath + "/" + filename)
                elif (filename.endswith(".xlsx")):
                    temp = pd.read_excel(dirpath + "/" + filename, 0)
                temp["bd_year"] = year
                if entregasDetalles is None:
                    entregasDetalles = temp
                else:
                    entregasDetalles = pd.concat([entregasDetalles, temp]) 

In [5]:
entregas["year"] = pd.to_numeric(entregas.FechaEnt.astype(str)\
            .map(lambda x: (x[0:4] if x.startswith("201") else x[-4:])), errors= "coerce")
entregas["month"] = pd.to_numeric(entregas.FechaEnt.astype(str)\
            .map(lambda x: (x[5:7] if x.startswith("201") else x[-7:-5])), errors= "coerce")

In [6]:
entregas.month.value_counts(dropna=False)

 6.0     21462
 11.0    20715
 10.0    20076
 12.0    16650
 5.0     16293
 4.0     14963
 9.0     14422
 8.0     13125
 7.0     12311
 3.0      5159
 2.0      4445
 1.0      3853
NaN        124
Name: month, dtype: int64

In [7]:
entregas[(entregas.year == 2016) & (entregas.bd_year == 2016)].CodDepto.value_counts()

16.0    2546
17.0    1998
5.0     1424
18.0    1194
11.0    1131
12.0     975
14.0     775
21.0     584
6.0      558
9.0      522
19.0     449
10.0     138
7.0       85
22.0      74
13.0      46
4.0       14
Name: CodDepto, dtype: int64

In [8]:
entregas[entregas.Pabellones.isna() ==False]

,Apellido,Apellido_2,AñoProy,Camas,Categoria,Categoría,Cedula,CodBoleta,CodComun,CodDepto,...,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,bd_year,lp,nomlp,year,month


In [9]:
entregas[(entregas.year == 2016) & (entregas.bd_year == 2017)].CodDepto.value_counts()

16.0    2546
17.0    1998
5.0     1424
18.0    1194
14.0     775
9.0      522
19.0     456
10.0     138
7.0       85
Name: CodDepto, dtype: int64

In [10]:
entregas.columns

Index(['Apellido', 'Apellido_2', 'AñoProy', 'Camas', 'Categoria', 'Categoría',
       'Cedula', 'CodBoleta', 'CodComun', 'CodDepto', 'CodMuni', 'CodReg',
       'FechaEnt', 'Firma', 'Huella', 'MEmbarazada', 'Menores5a', 'Nombre',
       'Nombre_2', 'OID_', 'Pabellones', 'Personas', 'ProyectoMalaria',
       'Responsable', 'Sustit', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',
       'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'bd_year', 'lp', 'nomlp',
       'year', 'month'],
      dtype='object')

In [17]:
entregas["uid"] = entregas.CodComun.astype(str) +entregas.CodBoleta.astype(str) +\
                    entregas.FechaEnt.astype(str)

In [29]:
entregas_dedup = entregas.drop_duplicates("uid", keep = 'first')

In [40]:
entregas_sub = entregas[((entregas.bd_year==2015) & (entregas.year <= 2015)) | 
                 ((entregas.bd_year==2016) & (entregas.year == 2016)) |
                 ((entregas.bd_year==2017) & (entregas.year == 2017)) |
                 ((entregas.bd_year==2018) & (entregas.year == 2018))]
entregas_sub = entregas_sub.drop_duplicates("uid", keep = 'first')

In [41]:
(entregas_sub.bd_year).value_counts().sort_index(), \
(entregas_dedup.bd_year).value_counts().sort_index()

(2015    36587
 2016    12173
 2017     2851
 2018     4573
 Name: bd_year, dtype: int64, 2015    37153
 2016    18861
 2017    11798
 2018     4631
 Name: bd_year, dtype: int64)

In [42]:
(entregas_sub.year).value_counts().sort_index(), \
(entregas_dedup.year).value_counts().sort_index()

(2012.0    12001
 2013.0     4519
 2014.0     2226
 2015.0    17841
 2016.0    12173
 2017.0     2851
 2018.0     4573
 Name: year, dtype: int64, 2012.0    19856
 2013.0     6210
 2014.0     3314
 2015.0    21901
 2016.0    13659
 2017.0     2852
 2018.0     4573
 Name: year, dtype: int64)

In [49]:
eMerge = pd.merge(entregas_dedup[["CodDepto", "CodComun", "CodBoleta", "bd_year", "FechaEnt", "year", "month"]], 
                  entregasDetalles[["CodDepto", "CodComun", "CodBoleta", "bd_year", 
                                    "Personas", "MEmbarazada", "Menores5a", "Camas", "Pabellones", "Sustit"]], 
                  how="inner", 
                  on=["CodComun", "CodBoleta", "bd_year"], suffixes=("_e", "_eD"))

In [45]:
eMerge.shape

(562366, 14)

In [46]:
entregasDetalles.shape

(1530109, 30)

In [55]:
entregas[entregas.CodComun.astype(str).map(lambda x: x[0]) != 
            entregas.CodDepto.astype(str).map(lambda x: x[0])]

,Apellido,Apellido_2,AñoProy,Camas,Categoria,Categoría,Cedula,CodBoleta,CodComun,CodDepto,...,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,bd_year,lp,nomlp,year,month,uid
1360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,12.0,...,1360.0,1360.0,NaN,NaN,2015,NaN,NaN,NaN,NaN,0.00nan
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,12.0,...,1.0,1.0,NaN,NaN,2016,NaN,NaN,NaN,NaN,0.00nan
1754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,12.0,...,1754.0,1754.0,NaN,NaN,2016,NaN,NaN,NaN,NaN,0.00nan
3186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,12.0,...,3186.0,3186.0,NaN,NaN,2016,NaN,NaN,NaN,NaN,0.00nan
3504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,12.0,...,3504.0,3504.0,NaN,NaN,2016,NaN,NaN,NaN,NaN,0.00nan


In [50]:
eMerge = eMerge[eMerge.year.isna() == False]

In [53]:
eMerge[eMerge.Sustit == False].groupby(["CodDepto_e", (eMerge.year)]).\
    Pabellones.sum().unstack(1).fillna(0)

year,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0
CodDepto_e,,,,,,,
2.0,0.0,0.0,0.0,5137.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,2500.0,346.0,0.0,0.0
5.0,63189.0,12661.0,15720.0,24268.0,25289.0,2103.0,5307.0
6.0,0.0,0.0,6243.0,8968.0,18250.0,0.0,0.0
7.0,0.0,0.0,0.0,6070.0,1936.0,422.0,0.0
9.0,0.0,0.0,0.0,19881.0,14228.0,446.0,1147.0
10.0,63792.0,47455.0,0.0,45613.0,57246.0,59.0,0.0
11.0,29612.0,28303.0,0.0,45595.0,28365.0,0.0,24320.0
12.0,19487.0,20463.0,50.0,25065.0,20500.0,0.0,0.0


In [46]:
deptos = gp.read_file("../../../../Covariates and Other Data/GIS/GT-IGN-cartografia_basica-Departamentos.geojson")
deptos = deptos[lambda x: x.CODIGO.astype(int) <= 2200]

In [ ]:
mlp.rcParams['figure.figsize'] = [10.0, 10.0]
mlp.rcParams['axes.titlesize'] = 20
temp = deptos.merge(eMerge.groupby("CodDepto_e").Pabellones.sum().divide(1000).reset_index()\
                         .assign(deptocode=lambda x: (x.CodDepto_e * 100).astype(int).astype(str)\
                                 .map(lambda y: y if y != 1 else "101")), 
                 left_on="CODIGO", right_on = "deptocode", how="outer" ).fillna(0)
ax = temp[temp.Pabellones == 0].plot(color="grey")
temp[temp.Pabellones > 0].plot("Pabellones", ax = ax, cmap = "Blues", legend=True, edgecolor="black")
ax.axis('off')
ax.set_title("Pabellones entregados entre 2012 y 2017 \n(en miles)")
    


In [64]:
pblYears = eMerge.groupby(["CodDepto_e", "year"]).Pabellones.sum().divide(1000).reset_index()\
                         .assign(deptocode=lambda x: (x.CodDepto_e * 100).astype(int).astype(str)\
                                 .map(lambda y: y if y != 1 else "101"))

In [ ]:
vmax = pblYears.Pabellones.max()

side = 8
n = 7
vmin = 0
#vmax = 8
f, axes = plt.subplots(int(np.ceil(n/2)), 2, figsize=(side*2*1.1, side*np.ceil(n/2) ))
i = 2012
for row in axes:
    for ax in row:
        temp3 = deptos.merge(pblYears[pblYears.year == i], 
                             left_on="CODIGO", right_on = "deptocode", how="left" ).fillna(0)
        temp3[temp3.Pabellones == 0].plot(ax=ax, color="grey")
        if len(temp3[temp3.Pabellones > 0])>0:
            temp3[temp3.Pabellones > 0].plot("Pabellones", ax = ax, cmap = "Blues", 
                                             legend=True, edgecolor="black",
                                             vmin = vmin, vmax = vmax)
        ax.axis('off')
        ax.set_title("Malaria bednets delivered\nThousands by department for "+ str(i))
        i += 1

In [66]:
eMerge.to_csv("../../../../Outcome Measurement Data/MALARIA/GTM - Bednets Distr.csv")

In [67]:
eMerge[(eMerge.year==2016) & (eMerge.CodDepto_e == 5)].Pabellones.sum()

41708.0

In [57]:
pblYears[pblYears.year==2018]

,CodDepto_e,year,Pabellones,deptocode
